In [8]:
import numpy as np
from scipy import optimize
from scipy.optimize import fsolve
import sympy as sm
import MalthusModel

model_a = MalthusModel.Malthus_analytical()
model_cobb = MalthusModel.Malthus_cobbd()

In [14]:
import sympy as sm

# Define symbolic variables
A, X, alpha, eta, L_t, mu = sm.symbols('A X alpha eta L_t mu')

# Define the equations
Yt = ((A * X)**(1 - alpha)) * (L_t)**alpha
nt = eta * (Yt / L_t)
Lt1 = nt * L_t + (1 - mu) * L_t
yt = Yt / L_t

# Define the condition to solve
condition = sm.Eq(L_t, Lt1)

# Solve the equation for L_t
solve = sm.solve(condition, L_t)

# Print the solution
solve


[(eta*(A*X)**(1 - alpha)/mu)**(-1/(alpha - 1))]

Finder SS værdier:

In [ ]:
eta = 1 #Definerer eta
L_guess = 10 #indstiller første gæt af L

initial_guess = [L_guess, model_cobb.Yt(L_guess)] #indstiller første gæt af L og Y

SS_values = fsolve(model_cobb.ss_conditions, initial_guess, args=(eta))

L_SS_value, Y_SS_value = SS_values

#print(SS_values)
print('The steady state value for the output Y is ', model_cobb.Yt(L_SS_value))
print('The steady state value for the labor force L is ', model_cobb.Lt1(L_SS_value, eta))

In [ ]:
import matplotlib.pyplot as plt
initial_L = 10  # Initial value of L
num_periods = 75  # Number of time periods to simulate

# Arrays to store values over time
L_values = np.zeros(num_periods)
Y_values = np.zeros(num_periods)

# Initialize the first value
L_values[0] = initial_L
Y_values[0] = model_cobb.Yt(L_values[0])

# Simulate the system over time
for t in range(1, num_periods):
    Y_t = Y_values[t - 1]
    L_t = L_values[t - 1]
    
    # Calculate n_t
    n_t = model_cobb.nt(eta, L_t)
    
    # Calculate L_{t+1}
    L_next = model_cobb.Lt1(L_t, eta)
    
    # Update arrays
    L_values[t] = L_next
    Y_values[t] = model_cobb.Yt(L_next)

# Plotting the results
time_periods = np.arange(num_periods)

plt.figure(figsize=(10, 6))
plt.plot(time_periods, L_values, label='Labor (L)')
plt.plot(time_periods, Y_values, label='Output (Y)')
plt.xlabel('Time (t)')
plt.ylabel('Values')
plt.title('Development of L and Y over Time')
plt.legend()
plt.grid(True)
plt.show()
